In [ ]:
# Corrección de cuantiles empírica.

import os
import sys

import numpy as np
import pandas as pd

import xarray as xr

i = sys.argv[1]
v = "UVHI"
dims = ["time", "lat", "lon"]

path_d    = f"../temp/NSRDB_4km/grid/WRF_1985_2014_4km_{i}.nc"
path_m    = f"../temp/WRF_1985_2014_4km/radiacion/WRF_1985_2014_4km_{i}.nc"
#path_f    = "../temp/WRF_2040_regrid_CHIRPS_days_cdf.nc"
path_res  = f"../temp/NSRDB_4km/map_res/WRF_regrid_CHIRPS_days_map_{i}.nc"
path_map  = f"../temp/NSRDB_4km/map/WRF_regrid_CHIRPS_days_qmap_{i}.nc"
#path_fmap = "../temp/WRF_2040_regrid_CHIRPS_days_qmap.nc"

sum = True

with xr.open_dataset(path_m) as ds_m:
    with xr.open_dataset(path_d) as ds_d:
        #with xr.open_dataset(path_f) as ds_f:

            df_d = ds_d.to_dataframe().sort_index()
            df_m = ds_m.to_dataframe().sort_index()
            #df_f = ds_f.to_dataframe().sort_index()
            #df_f["map"] = None
            df_m["map"] = None
            df_m["diff_sum"] = None
            #df_m["diff_div"] = None

            latitude = df_d.index.get_level_values(dims[1]).unique()
            longitude = df_d.index.get_level_values(dims[2]).unique()

            for lat in latitude:
                print(f"{lat:.3f}°N")
                for lon in longitude:
                    df_xs_m = df_m.loc[ (slice(None), lat, lon),
                        df_m.columns ].sort_values("q_model")
                    df_xs_d = df_d.loc[ (slice(None), lat, lon),
                        df_d.columns ].sort_values("q_model")
                    df_xs_m["map"] = np.interp( df_xs_m["q_model"].values,
                        df_xs_d["q_model"].values, df_xs_d[v].values )
                    df_xs_m["map"] = df_xs_m["map"].where(df_xs_m["map"]>0, 0)
                    df_xs_m["diff_sum"] = df_xs_m["map"] - df_xs_m[v]
                    df_m.loc[ (slice(None), lat, lon), df_m.columns ] = df_xs_m
                    #df_xs_m["diff_div"] = df_xs_m["map"] / df_xs_m[v]

                    #df_xs_f = df_f.loc[ (slice(None), lat, lon), 
                    #    df_f.columns ].sort_values("q_model")
                    #if sum:
                    #df_xs_f["map"] = np.interp( df_xs_f["q_model"].values,
                    #    df_xs_m["q_model"].values,
                    #    df_xs_m["diff_sum"].values )
                    #df_xs_f["map"] = df_xs_f["map"] + df_xs_f[v]
                    #else:
                    #    df_xs_f["map"] = np.interp( df_xs_f["q_model"].values,
                    #        df_xs_m["q_model"].values,
                    #        df_xs_m["diff_div"].values )
                    #    df_xs_f["map"] = df_xs_f["map"] * df_xs_f[v]
                    #df_xs_f["map"] = df_xs_f["map"].where(df_xs_f["map"]>0, 0)
                    #df_f.loc[ (slice(None), lat, lon), df_f.columns ] = df_xs_f

            ds_map = df_m.to_xarray()
            ds_m["map"] = ( dims, ds_map["map"].values )
            ds_m["diff_sum"] = ( dims, ds_map["diff_sum"].values )
            #ds_m["diff_div"] = ( dims, ds_map["diff_div"].values )
            ds_m.to_netcdf( path_res )
            ds_m[["map"]].rename( {"map": v} ).to_netcdf( path_map )

            #ds_fmap = df_f.to_xarray()
            #ds_f["map"] = ( dims, ds_fmap["map"].values )
            #ds_f[["map"]].rename( {"map": v} ).to_netcdf( path_fmap )

In [ ]:
# Paso 3.
# Mapeo de cuantiles empírico.

import numpy as np
import pandas as pd

import xarray as xr

In [ ]:
# Rutas de archivos.
path_d = "/Users/rodrigo/Desktop/zzR_zz_Mega_Variables_Extraidas.nc"
path_r = "/Users/rodrigo/Desktop/zzR_zz_Mega_Variables_Extraidas_2.nc"
path_m = "/Users/rodrigo/Desktop/zzR_zz_Mega_Variables_Extraidas_2.nc"
path_q = "/Users/rodrigo/Desktop/zzR_zz_Mega_Variables_Extraidas_2.nc"

# Datos medidos.
ds_d = xr.open_dataset( path_d )
# Modelo histórico original.
ds_r = xr.open_dataset( path_r )
# Modelo histórico corregido, copia del archivo del modelo histórico original.
ds_m = xr.open_dataset( path_r )
# Mapeo de cuantiles, calculado en pasos anteriores.
ds_q = xr.open_dataset( path_r )

In [ ]:
# Creamos el archivo de mapeo de cuantiles.

# Rutas de archivos.
path_r = "/Users/rodrigo/Desktop/zzR_zz_Mega_Variables_Extraidas_2.nc"
path_q = "/Users/rodrigo/Desktop/zzR_zz_Mega_Variables_Extraidas_2.nc"

# Abrimos el archivo.
with xr.open_dataset( path_r ) as ds:

    # Cambiamos tiempo por la distribución acumulada.
    CDF = np.linspace( 1/ds_r["time"].shape[0], 1, ds_r["time"].shape[0] )
    ds["time"] = CDF

    # Guardamos le archivo.
    ds.to_netcdf(path_q)

In [ ]:
# Procesamos para todas las latitudes y longitudes.
for lat in ds_d["lat"]:
    print(f"Procesando latitud {lat:.2f}...")
    for lon in ds_d["lon"]:

        df_d = ds_d.sel( {"lat": lat, "lon": lon} ).to_dataframe(
            ).reset_index().set_index("time")
        df_r = ds_r.sel( {"lat": lat, "lon": lon} ).to_dataframe(
            ).reset_index().set_index("time")

        # Creamos DataFrames de apoyo.
        df_m = df_r.copy()
        df_q = pd.DataFrame( index = np.linspace(
            1/df_r.shape[0], 1, df_r.shape[0] ),
            columns = df_r.columns )
        df_q = df_q.rename_axis("CDF", axis = 0)

        # Inicializamos las variables como 0.
        df_m[ ["GHI", "DNI"] ] = 0

        # Iteramos para todas las columnas.
        for v in df_d.columns:
            # Ordenamos los valores originales y destino incluyendo el tiempo.
            if v in ["DNI", "GHI"]:
                df_i = df_d[[v]].where( df_d[[v]] > 0, np.nan
                    ).dropna().sort_values(v).reset_index()
                df_j = df_r[[v]].where( df_r[[v]] > 0, np.nan
                    ).dropna().sort_values(v).reset_index()
            else:
                df_i = df_d[[v]].sort_values(v).reset_index()
                df_j = df_r[[v]].sort_values(v).reset_index()
            # Calculamos la distribución acumulada.
            df_i["CDF"] = np.linspace( 1/df_i.shape[0], 1, df_i.shape[0] )
            df_j["CDF"] = np.linspace( 1/df_j.shape[0], 1, df_j.shape[0] )

            # Interpolamos los valores de la distribución origen a la destino.
            df_j["Map"] = np.interp( df_j["CDF"], df_i["CDF"],
                df_i[v] ).round( decimals = 2 )
            df_j["Quantile"] =  df_j["Map"] - df_j[v]
            df_j = df_j.set_index("time").sort_index()
            # Reordenamos los valores.
            df_m.loc[ df_j.index, v ] = df_j["Map"]
            # Guardamos el mapeo de cuantiles para la corrida futura.
            df_j = df_j.set_index("CDF").sort_index()
            if v in ["DNI", "GHI"]:
                q = np.interp( df_q.index, df_j.index, 
                    df_j["Quantile"].values )
            else: q = df_j["Quantile"].values
            df_q[v] = q

        # Nos aseguramos que los datos estén dentro de su rango.
        df_m["Relative Humidity"] = df_m["Relative Humidity"].where(
            df_m["Relative Humidity"] > 0, 0 )
        df_m["Relative Humidity"] = df_m["Relative Humidity"].where(
            df_m["Relative Humidity"] < 100, 100 )
        df_m["Wind Direction"] = df_m["Wind Direction"].where(
            df_m["Wind Direction"] < 360, df_m["Wind Direction"] - 360 )
        df_m["Wind Speed"] = df_m["Wind Speed"].where(
            df_m["Wind Speed"] > 0 , 0 )
        df_m["GHI"] = df_m["GHI"].where( df_m["GHI"] > 0 , 0 )
        df_m["DNI"] = df_m["DNI"].where( df_m["DNI"] > 0 , 0 )

        # Asignamos al Dataset.
        ds_m.loc[ {"lat": lat, "lon": lon} ] = df_m.reset_index().set_index(
            ["time", "lat", "lon"] ).astype(float).to_xarray()
        ds_q.loc[ {"lat": lat, "lon": lon} ] = df_q.reset_index(
            ).set_index( ["CDF", "lat", "lon"] ).astype(float).to_xarray()
        
# Guardamos los archivos.
ds_m = ds_m.to_netcdf(path_m, mode = "w")
ds_q = ds_m.to_netcdf(path_q, mode = "w")